## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '../..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://github.com/anhaidgroup/deepmatcher/blob/master/Datasets.md

In [6]:
import glob
import csv
from tqdm.auto import tqdm

from entity_embed.data_utils.utils import Enumerator

row_dict = {}
id_enumerator = Enumerator()
left_id_set = set()
right_id_set = set()
rows_total = 2554 + 22074
clusters_total = 1154

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/tableA.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'left-{int(row["id"])}']
            row['source'] = 'left'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/tableB.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'right-{int(row["id"])}']
            row['source'] = 'right'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [7]:
train_true_pair_set = set()
valid_true_pair_set = set()
test_true_pair_set = set()

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/train.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            train_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/valid.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            valid_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/test.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            test_true_pair_set.add((id_left, id_right))
        
display(('train_true_pair_set', len(train_true_pair_set)))
display(('valid_true_pair_set', len(valid_true_pair_set)))
display(('test_true_pair_set', len(test_true_pair_set)))

('train_true_pair_set', 576)

('valid_true_pair_set', 193)

('test_true_pair_set', 193)

## Preprocess

In [8]:
attr_list = ['title', 'category', 'brand', 'modelno', 'price']

In [9]:
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'title': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'category': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'brand': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'modelno': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [13]:
from entity_embed import build_row_numericalizer

row_numericalizer = build_row_numericalizer(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

14:45:06 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
14:45:10 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
14:45:14 INFO:For attr='brand', computing actual alphabet and max_str_len
14:45:14 INFO:For attr='brand', using actual_max_str_len=48
14:45:14 INFO:For attr='modelno', computing actual alphabet and max_str_len
14:45:14 INFO:For attr='modelno', using actual_max_str_len=48
14:45:14 INFO:For attr='price', computing actual alphabet and max_str_len
14:45:14 INFO:For attr='price', using actual_max_str_len=10


{'title': NumericalizeInfo(field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer=<function default_tokenizer at 0x7faa22b20dc0>, alphabet=None, max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7fa9fa8b8f10>),
 'category': NumericalizeInfo(field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer=<function default_tokenizer at 0x7faa22b20dc0>, alphabet=None, max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7fa9fa8b81c0>),
 'brand': NumericalizeInfo(field_type=<FieldType.STRING: 'string'>, tokenizer=<function default_tokenizer at 0x7faa22b20dc0>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=48, vocab=None),
 

In [14]:
from entity_embed import PairwiseDataModule

datamodule = PairwiseDataModule(
    row_dict=row_dict,
    row_numericalizer=row_numericalizer,
    batch_size=10,
    row_batch_size=16,
    train_true_pair_set=train_true_pair_set,
    valid_true_pair_set=valid_true_pair_set,
    test_true_pair_set=test_true_pair_set,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    use_mask=True,
    embedding_size=300
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = 'f1-walmart-amazon'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

14:45:14 INFO:GPU available: True, used: True
14:45:14 INFO:TPU available: None, using: 0 TPU cores
14:45:14 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

14:45:16 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 18.5 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
9.6 M     Trainable params
8.9 M     Non-trainable params
18.5 M    Total params


14:45:17 WARNING:true_pair_set has 38 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [18]:
model.blocker_net.get_signature_weights()

{'title': 0.4002133309841156,
 'category': 0.021198181435465813,
 'brand': 0.2585507929325104,
 'modelno': 0.2179427295923233,
 'price': 0.10209502279758453}

## Testing

In [19]:
trainer.test(ckpt_path='best')


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1_at_0.3': 0.5444287729196051,
 'test_f1_at_0.5': 0.7914893617021276,
 'test_f1_at_0.7': 0.8759894459102902,
 'test_f1_at_0.9': 0.5818181818181818,
 'test_pair_entity_ratio_at_0.3': 1.34375,
 'test_pair_entity_ratio_at_0.5': 0.7213541666666666,
 'test_pair_entity_ratio_at_0.7': 0.484375,
 'test_pair_entity_ratio_at_0.9': 0.21354166666666666,
 'test_precision_at_0.3': 0.374031007751938,
 'test_precision_at_0.5': 0.6714801444043321,
 'test_precision_at_0.7': 0.8924731182795699,
 'test_precision_at_0.9': 0.975609756097561,
 'test_recall_at_0.3': 1.0,
 'test_recall_at_0.5': 0.9637305699481865,
 'test_recall_at_0.7': 0.8601036269430051,
 'test_recall_at_0.9': 0.41450777202072536}
--------------------------------------------------------------------------------


[{'test_precision_at_0.3': 0.374031007751938,
  'test_recall_at_0.3': 1.0,
  'test_f1_at_0.3': 0.5444287729196051,
  'test_pair_entity_ratio_at_0.3': 1.34375,
  'test_precision_at_0.5': 0.6714801444043321,
  'test_recall_at_0.5': 0.9637305699481865,
  'test_f1_at_0.5': 0.7914893617021276,
  'test_pair_entity_ratio_at_0.5': 0.7213541666666666,
  'test_precision_at_0.7': 0.8924731182795699,
  'test_recall_at_0.7': 0.8601036269430051,
  'test_f1_at_0.7': 0.8759894459102902,
  'test_pair_entity_ratio_at_0.7': 0.484375,
  'test_precision_at_0.9': 0.975609756097561,
  'test_recall_at_0.9': 0.41450777202072536,
  'test_f1_at_0.9': 0.5818181818181818,
  'test_pair_entity_ratio_at_0.9': 0.21354166666666666}]

## Testing manually 

In [20]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [21]:
embedding_size = model.blocker_net.embedding_size

In [22]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [23]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 325 ms, sys: 1.02 ms, total: 326 ms
Wall time: 42.9 ms


In [24]:
%%time

sim_threshold = 0.7
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 466 ms, sys: 0 ns, total: 466 ms
Wall time: 50.6 ms


In [25]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

0.484375

In [26]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, test_true_pair_set)
precision, recall

(0.8924731182795699, 0.8601036269430051)

In [27]:
f1_score(precision, recall)

0.8759894459102902

In [28]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

20

In [29]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

27

In [30]:
cos_similarity = lambda a, b: np.dot(a, b)

In [31]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.6917216,
 {'id': 932,
  'title': 'da - lite dual vision tensioned cosmopolitan electrol - av format 10 x 10 diagonal',
  'category': 'electronics - general',
  'brand': 'da - lite',
  'modelno': '84992',
  'price': '2735 . 99',
  'source': 'left'},
 {'id': 5367,
  'title': 'da - lite tensioned cosmopolitan electrol - projection screen motorized - 1 1 - dual vision',
  'category': 'projection screens',
  'brand': 'da - lite',
  'modelno': 'cosmopolitan electrol',
  'price': '',
  'source': 'right'})

(0.6514168,
 {'id': 1613,
  'title': 'pinnacle studio hd v15 ultimate collection pc',
  'category': 'software',
  'brand': 'avid technology',
  'modelno': '',
  'price': '119 . 0',
  'source': 'left'},
 {'id': 19772,
  'title': 'pinnacle studio ultimate collection v . 15',
  'category': 'computers accessories',
  'brand': 'avid',
  'modelno': '82103002401',
  'price': '99 . 99',
  'source': 'right'})

(0.69834864,
 {'id': 1340,
  'title': 'cta mini battery chargers for nikon en - el9 digital cameras',
  'category': 'electronics - general',
  'brand': 'cta',
  'modelno': 'mr - enel9',
  'price': '15 . 98',
  'source': 'left'},
 {'id': 11050,
  'title': 'cta mr - enel9 mini battery charger kit for nikon en - el9 battery',
  'category': 'battery chargers',
  'brand': 'cta digital',
  'modelno': 'mr - enel9',
  'price': '8 . 32',
  'source': 'right'})

(0.6242144,
 {'id': 1810,
  'title': 'microsoft comfort optical mouse 3000 silver blue',
  'category': 'mice',
  'brand': 'microsoft',
  'modelno': 'd1t00011',
  'price': '19 . 95',
  'source': 'left'},
 {'id': 3131,
  'title': 'comfort opt mse3000 silver blu',
  'category': 'computers accessories',
  'brand': 'microsoft',
  'modelno': 'd1t - 00011',
  'price': '11 . 99',
  'source': 'right'})

(0.48696736,
 {'id': 2315,
  'title': 'seiko slp - mrl multipurpose label',
  'category': 'stationery & office machinery',
  'brand': 'seiko',
  'modelno': 'slp - mrl',
  'price': '14 . 88',
  'source': 'left'},
 {'id': 12102,
  'title': 'smart label printer multipurpose labels',
  'category': 'audio video accessories',
  'brand': 'seiko',
  'modelno': '',
  'price': '13 . 82',
  'source': 'right'})

(0.42587325,
 {'id': 1396,
  'title': 'dj - tech t545a 15 600w 2 - way active loudspeaker',
  'category': 'stereos / audio',
  'brand': 'dj tech',
  'modelno': 't545a',
  'price': '358 . 0',
  'source': 'left'},
 {'id': 4992,
  'title': 'djtech super powered 15 600 w dj spkrs',
  'category': 'home audio theater',
  'brand': 'dj - tech',
  'modelno': 't545a',
  'price': '396 . 74',
  'source': 'right'})

(0.59969115,
 {'id': 2118,
  'title': 'antec tricool 92mm double ball bearing case fan',
  'category': 'electronics - general',
  'brand': 'antec',
  'modelno': 'tricool 92mm dbb',
  'price': '8 . 25',
  'source': 'left'},
 {'id': 23175,
  'title': 'antec tricool 92mm dbb case fan with 3 - spd switch 3 4 - pin 3 - pin monitoring',
  'category': '',
  'brand': 'antec',
  'modelno': 'tricool 92mm dbb',
  'price': '8 . 25',
  'source': 'right'})

(0.5219086,
 {'id': 992,
  'title': 'hp 61 black tri - color combo inkjet cartridge',
  'category': 'printers',
  'brand': 'hp',
  'modelno': 'hp 61',
  'price': '30 . 5',
  'source': 'left'},
 {'id': 22146,
  'title': 'hp 61 ink cartridge combo pack',
  'category': 'inkjet printer ink',
  'brand': 'hp',
  'modelno': 'cr259fn # 140',
  'price': '28 . 2',
  'source': 'right'})

(0.5870779,
 {'id': 88,
  'title': 'balt wheasel easel adjustable melamine dry erase board white',
  'category': 'stationery & office machinery',
  'brand': 'balt',
  'modelno': '33250',
  'price': '239 . 88',
  'source': 'left'},
 {'id': 5823,
  'title': 'balt inc . wheasel easel adjustable melamine dry erase board 28 3 4 x 59 1 2 white',
  'category': 'laminating supplies',
  'brand': 'mayline',
  'modelno': '',
  'price': '134 . 45',
  'source': 'right'})

(0.50563633,
 {'id': 1549,
  'title': 'comprehensive 1 4 standard phone plug to 3 - pin xlr jack set of 10',
  'category': 'tv accessories',
  'brand': 'comprehensive',
  'modelno': 'spp - xlrj',
  'price': '79 . 9',
  'source': 'left'},
 {'id': 21502,
  'title': 'comprehensive premium - audio adapter - mini - phone 3 . 5 mm m - 3 pin xlr f',
  'category': 'connectors adapters',
  'brand': 'comprehensive video',
  'modelno': '',
  'price': '',
  'source': 'right'})